# Підключення до БД та SQL запит

In [1]:
!pip install --upgrade google-cloud-bigquery

# (UA) Імпортуємо необхідні бібліотеки для роботи з даними та виконання статистичних тестів
# (ENG) Import required libraries for data manipulation and statistical tests
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# (UA) Аутентифікація користувача для доступу до Google Cloud
# (ENG) Authenticate user to access Google Cloud
auth.authenticate_user()

# (UA) Створення клієнта для доступу до BigQuery
# (ENG) Create a client for accessing BigQuery
client = bigquery.Client(project="data-analytics-mate")

# (UA) SQL-запит для вибірки даних з бази даних
# (ENG) SQL query to fetch data from the database
query = """
with session_info as (
  select
        s.date,
        s.ga_session_id,
        sp.country,
        sp.device,
        sp.continent,
        sp.channel,
        ab.test,
        ab.test_group
  from `DA.ab_test` ab
  join `DA.session` s
  on ab.ga_session_id = s.ga_session_id
  join `DA.session_params` sp
  on sp.ga_session_id = ab.ga_session_id
),
session_with_orders as (
  select
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        count(distinct o.ga_session_id) as session_with_orders
  from `DA.order` o
  join session_info
  on o.ga_session_id = session_info.ga_session_id
  group by
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group
),events as (
  select
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        sp.event_name,
        count(sp.ga_session_id) as event_cnt
  from `DA.event_params` sp
  join session_info
  on sp.ga_session_id = session_info.ga_session_id
  group by
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        sp.event_name
),session as (
  select
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        count(distinct session_info.ga_session_id) as session_cnt
  from session_info
  group by
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group
), account as (
  select
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        count(distinct acs.ga_session_id) as new_account_cnt


  from `DA.account_session` acs
  join session_info
  on acs.ga_session_id = session_info.ga_session_id
  group by
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group
)
  select
        session_with_orders.date,
        session_with_orders.country,
        session_with_orders.device,
        session_with_orders.continent,
        session_with_orders.channel,
        session_with_orders.test,
        session_with_orders.test_group,
        'session_with_orders' as event_name,
        session_with_orders.session_with_orders as value
  from session_with_orders
  union all
 select
        events.date,
        events.country,
        events.device,
        events.continent,
        events.channel,
        events.test,
        events.test_group,
        event_name,
        event_cnt as value
  from events
  union all
 select
        session.date,
        session.country,
        session.device,
        session.continent,
        session.channel,
        session.test,
        session.test_group,
        'session' as event_name,
        session_cnt as value
  from session
  union all
 select
        account.date,
        account.country,
        account.device,
        account.continent,
        account.channel,
        account.test,
        account.test_group,
        'new_account' as event_name,
        new_account_cnt as value
  from account
"""

# (UA) Виконання SQL-запиту до BigQuery та отримання результатів
# (ENG) Execute SQL query in BigQuery and fetch the results
query_job = client.query(query)  # Виконання SQL-запиту
results = query_job.result()  # Очікування завершення запиту

# (UA) Перетворення результатів запиту на DataFrame для подальшого аналізу
# (ENG) Convert the query results to a DataFrame for further analysis
df = results.to_dataframe()

# (UA) Виведення DataFrame для перевірки результатів
# (ENG) Display DataFrame to check the results
df

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new_account,1
1,2020-11-02,Venezuela,mobile,Americas,Organic Search,2,2,new_account,1
2,2020-11-03,Cambodia,desktop,Asia,Direct,2,1,new_account,1
3,2020-11-03,Malta,desktop,Europe,Direct,2,1,new_account,1
4,2020-11-03,Estonia,mobile,Europe,Direct,2,1,new_account,1
...,...,...,...,...,...,...,...,...,...
800991,2020-12-05,Vietnam,desktop,Asia,Direct,3,1,user_engagement,1
800992,2020-12-17,Vietnam,mobile,Asia,Organic Search,4,1,user_engagement,1
800993,2020-12-19,Vietnam,desktop,Asia,Paid Search,4,2,select_promotion,1
800994,2020-12-26,Vietnam,desktop,Asia,Paid Search,4,1,page_view,1


# Тестування

## Групуємо дані для подальшої роботи

In [ ]:
# (UA) Крок 1: Агрегація сирих даних
# (ENG) Step 1: Aggregation of raw data

# (UA) Групуємо DataFrame 'df' за трьома ключовими колонками:
# (UA) 'test', 'test_group' та 'event_name'.
# (ENG) Group the 'df' DataFrame by three key columns:
# (ENG) 'test', 'test_group', and 'event_name'.

# (UA) Для кожної унікальної групи беремо колонку 'value' і рахуємо суму (.sum()).
# (ENG) For each unique group, take the 'value' column and calculate the sum (.sum()).

# (UA) .reset_index() перетворює згруповані колонки (які стали індексом)
# (UA) назад у звичайні колонки для зручності.
# (ENG) .reset_index() converts the grouped columns (which became the index)
# (ENG) back into regular columns for convenience.
agg_df = df.groupby(['test', 'test_group', 'event_name'])['value'].sum().reset_index()

# (UA) Виведемо перші 10 рядків, щоб перевірити результат
# (ENG) Print the first 10 rows to check the result
print(agg_df.head(10))

   test  test_group         event_name   value
0     1           1   add_payment_info    1988
1     1           1  add_shipping_info    3034
2     1           1        add_to_cart    1395
3     1           1     begin_checkout    3784
4     1           1              click     368
5     1           1        first_visit   30596
6     1           1        new_account    3823
7     1           1          page_view  191543
8     1           1             scroll   73244
9     1           1        select_item     543


## Робимо список з метрик

In [ ]:
# (UA) Крок 2: Визначення списку метрик для розрахунку
# (ENG) Step 2: Defining the list of metrics for calculation

# (UA) Визначаємо список чисельників (подій, що нас цікавлять)
# (ENG) Define the list of numerators (events of interest)
numerator_metrics = [
    'add_payment_info',
    'add_shipping_info',
    'begin_checkout',
    'new_account' ]

# (UA) Визначаємо знаменник (базу, на яку ділимо)
# (ENG) Define the denominator (the base we divide by)
denominator_metric = 'session'

## Розрахунки

In [ ]:
# (UA) Робимо "широку" таблицю з 'agg_df'.
# (UA) 'index' - стануть рядками.
# (UA) 'columns' - стануть колонками (1 -> cc, 2 -> rr).
# (UA) 'fill_value=0' - наш "безпечний" метод, який замінює NaN на 0.
# (ENG) Make a "wide" table from 'agg_df'.
# (ENG) 'index' - will become rows.
# (ENG) 'columns' - will become columns (1 -> cc, 2 -> rr).
# (ENG) 'fill_value=0' - is our "safe" method, replacing NaN with 0.
data_wide = agg_df.pivot_table(
    index=['test', 'event_name'],
    columns='test_group',
    values='value',
    fill_value=0
).reset_index()

# (UA) Перейменовуємо колонки 1 і 2 для зручності
# (ENG) Rename columns 1 and 2 for convenience
data_wide.rename(columns={1: 'cc_value', 2: 'rr_value'}, inplace=True)

# --- (UA)  Об'єднання ---
# --- (ENG)  Merge ---

# (UA) Створюємо окрему таблицю лише зі знаменниками
# (ENG) Create a separate table with only denominators
denominators = data_wide[data_wide['event_name'] == denominator_metric]
denominators = denominators[['test', 'cc_value', 'rr_value']] # Беремо лише потрібні колонки
denominators.rename(columns={'cc_value': 'denominator_cc', 'rr_value': 'denominator_rr'}, inplace=True)

# (UA) Створюємо окрему таблицю лише з чисельниками
# (ENG) Create a separate table with only numerators
numerators = data_wide[data_wide['event_name'].isin(numerator_metrics)]
numerators.rename(columns={'cc_value': 'numerator_cc', 'rr_value': 'numerator_rr', 'event_name': 'metric'}, inplace=True)

# (UA) Об'єднуємо чисельники та їхні знаменники за 'test'
# (ENG) Merge numerators and their denominators on 'test'
final_data = pd.merge(numerators, denominators, on='test')

# --- (UA)  Apply (Застосування розрахунків) ---
# --- (ENG)  Apply (Applying calculations) ---

# (UA) Розрахунок конверсій
# (ENG) Calculate conversions
final_data['conversion_cc'] = (final_data['numerator_cc'] / final_data['denominator_cc']).fillna(0)
final_data['conversion_rr'] = (final_data['numerator_rr'] / final_data['denominator_rr']).fillna(0)

# (UA) Створюємо функцію, яку будемо застосовувати до кожного рядка
# (ENG) Create a function to be applied to each row
def calculate_ztest(row):
    # (UA) Збираємо дані з поточного рядка
    # (ENG) Get data from the current row
    count = [row['numerator_cc'], row['numerator_rr']]
    nobs = [row['denominator_cc'], row['denominator_rr']]

    # (UA) Безпечна перевірка, якщо знаменник 0
    # (ENG) Safe check if denominator is 0
    if nobs[0] == 0 or nobs[1] == 0:
        return pd.Series([None, None])

    z_stat, p_value = proportions_ztest(count=count, nobs=nobs, alternative='two-sided')
    return pd.Series([z_stat, p_value])

# (UA) Застосовуємо функцію до кожного рядка (axis=1)
# (UA) і створюємо 2 нові колонки з результатами
# (ENG) Apply the function to each row (axis=1)
# (ENG) and create 2 new columns with the results
final_data[['metric_chang_z_stat', 'p_value']] = final_data.apply(calculate_ztest, axis=1)

# (UA) Розраховуємо значущість
# (ENG) Calculate significance
final_data['significant'] = final_data['p_value'] < 0.05

# (UA) Форматуємо назву метрики та колонки 'test'
# (ENG) Format the metric name and 'test' column
final_data['metric'] = final_data['metric'] + ' / ' + denominator_metric
final_data.rename(columns={'test': 'test_number'}, inplace=True)

# (UA) Використовуємо 'final_data' як наш фінальний DataFrame
# (ENG) Use 'final_data' as our final DataFrame
final_results_df = final_data

# (UA) Виводимо фінальну таблицю на екран
# (ENG) Display the final table
print("Розрахунки  завершено:")
print("Calculations  complete:")
display(final_results_df)

Розрахунки  завершено:
Calculations  complete:


/tmp/ipython-input-3198640618.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerators.rename(columns={'cc_value': 'numerator_cc', 'rr_value': 'numerator_rr', 'event_name': 'metric'}, inplace=True)


test_group,test_number,metric,numerator_cc,numerator_rr,denominator_cc,denominator_rr,conversion_cc,conversion_rr,metric_chang_z_stat,p_value,significant
0,1,add_payment_info / session,1988.0,2229.0,45362.0,45193.0,0.043825,0.049322,-3.924884,0.000087,True
1,1,add_shipping_info / session,3034.0,3221.0,45362.0,45193.0,0.066884,0.071272,-2.603571,0.009226,True
2,1,begin_checkout / session,3784.0,4021.0,45362.0,45193.0,0.083418,0.088974,-2.978783,0.002894,True
3,1,new_account / session,3823.0,3681.0,45362.0,45193.0,0.084278,0.081451,1.542883,0.122859,False
4,2,add_payment_info / session,2344.0,2409.0,50637.0,50244.0,0.04629,0.047946,-1.240994,0.214608,False
5,2,add_shipping_info / session,3480.0,3510.0,50637.0,50244.0,0.068724,0.069859,-0.709557,0.477979,False
6,2,begin_checkout / session,4262.0,4313.0,50637.0,50244.0,0.084168,0.085841,-0.952898,0.340642,False
7,2,new_account / session,4165.0,4184.0,50637.0,50244.0,0.082252,0.083274,-0.588793,0.556000,False
8,3,add_payment_info / session,3623.0,3697.0,70047.0,70439.0,0.051722,0.052485,-0.643172,0.520112,False
9,3,add_shipping_info / session,5298.0,5188.0,70047.0,70439.0,0.075635,0.073652,1.413727,0.157442,False


# Єкспорт результатів

In [ ]:
# (UA)  Експорт результатів у CSV-файл
# (ENG)  Export results to a CSV file

# (UA) Назва файлу
# (ENG) File name
file_name = "ab_test_results.csv"

# (UA) Зберігаємо DataFrame у CSV-файл
# (ENG) Save the DataFrame to a CSV file
final_results_df.to_csv(file_name, index=False)

print(f"Файл '{file_name}' успішно створено!")
print(f"File '{file_name}' created successfully!")

Файл 'ab_test_results.csv' успішно створено!
File 'ab_test_results.csv' created successfully!


# Посилання

[Дашброд](https://public.tableau.com/views/CreateYourABTestingTool_17617488603990/Dashboard1?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

[Google sheets](https://docs.google.com/spreadsheets/d/1kf2bUgpZgV1lVI3cAvQmsspU6iiBPmfe9NKqcFkC-qI/edit?usp=sharing)